In [1]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px
import datetime as dt

In [2]:
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
data = pd.read_csv(url)
data = data[data["continent"].notna()]
data['date'] = pd.to_datetime(data['date'])
data = data[['date','continent','iso_code','total_cases','total_deaths','total_cases_per_million','total_deaths_per_million']].fillna(0)

In [3]:
dates = pd.date_range(start="2020-01-03",end=pd.Timestamp.today())
marks = pd.date_range(start="2020-01-03",end=pd.Timestamp.today(),freq="MS")

In [4]:
continents= data["continent"].unique().tolist()
continents.append("World")
continents.remove("Oceania")

In [7]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(children='Világtérkép a COVID-19-ről', style={'textAlign': 'center', 'fontSize': 48}),
    html.Hr(),
    html.Div(
    [
        html.Label(children='Válasszon egy opciót:'),
        dcc.Dropdown(id='dropdown_col',
                options=[
                    {'label': 'Összes eset', 'value': 'total_cases'},
                    {'label': 'Összes haláleset', 'value': 'total_deaths'},
                    {'label': 'Összes eset 1 millió főre vetítve', 'value': 'total_cases_per_million'},
                    {'label': 'Összes haláleset 1 millió főre vetítve', 'value': 'total_deaths_per_million'}],
                value='total_cases'),
        html.Br(),
        html.Label(children='Válasszon egy földrészt:'),
        dcc.Dropdown(id='dropdown_cont',options=continents,
                    value="World")
    ],style={'width':'30%'}),
    dcc.Slider(min=0,
            max=len(marks)-1,
            value=0,
            marks={i : str(yearmon) for i, yearmon in enumerate(marks.strftime("%Y-%m"))},
            step=1,
            id='dateslider'),
    html.Div(id='label',style={'fontSize': 24}),
    dcc.Graph(id='graph_sc_geo',figure={}),
])


@callback(
    Output('graph_sc_geo', 'figure'),
    Output('label','children'),
    Input('dropdown_col', 'value'),
    Input('dateslider','value'),
    Input('dropdown_cont','value'),
    State('dropdown_col','options')
)
def update_graph(col,month,cont,text):
    df = data[data['date']==marks[month]]
    label = [x['label'] for x in text if x['value']==col][0]
    label_value = int(df[col].sum()) if cont == 'World' else int(df[df['continent']==cont][col].sum())
    text = f'{label}: {label_value:,}'
    fig = px.scatter_geo(df,
                         locations="iso_code",
                         size=col,
                         scope=cont.lower(),
                         size_max=40,
                         height=800,
                         color="continent"
                         )
    fig.update_geos(showcountries=True)
    return fig,text

                                                 
app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

